In [1]:
import pandas as pd

In [2]:
properati = pd.read_csv('datos/new.csv',index_col=0,error_bad_lines=False)
properati.columns

Index([u'created_on', u'operation', u'property_type', u'place_name',
       u'place_with_parent_names', u'country_name', u'state_name',
       u'geonames_id', u'lat-lon', u'lat', u'lon', u'price', u'currency',
       u'price_aprox_local_currency', u'price_aprox_usd',
       u'surface_total_in_m2', u'surface_covered_in_m2', u'price_usd_per_m2',
       u'price_per_m2', u'floor', u'rooms', u'expenses', u'properati_url',
       u'description', u'title', u'image_thumbnail'],
      dtype='object')

In [3]:
print len(properati)
nuevo = properati.loc[(properati.state_name.str.contains("Capital Federal")) | (properati.state_name.str.contains("G.B.A"))]
print len(nuevo)

121220
81150


In [4]:
nuevo = nuevo.dropna(subset=['price'])
nuevo.drop_duplicates(inplace=True)
nuevo.drop_duplicates(subset=['properati_url'],inplace=True)

def recalcular_precios(row):
    if (row.currency == 'USD'):
        row.price_aprox_usd   = float(row.price)
        row.price_aprox_local_currency  = row.price_aprox_usd * 17.6445
        
    if (row.currency == 'ARS'):
        row.price_aprox_local_currency  = float(row.price)
        row.price_aprox_usd  = row.price_aprox_local_currency // 17.6445
        
    if (row.surface_total_in_m2 and row.price_aprox_local_currency and row.price_aprox_usd):
        row.price_per_m2   = row.price_aprox_local_currency  // row.surface_total_in_m2
        row.price_usd_per_m2  = row.price_aprox_usd // row.surface_total_in_m2
    return row
nuevo = nuevo.apply(recalcular_precios,axis=1)
print len(nuevo)

72494


In [10]:
nuevo = nuevo.dropna(subset= ['lat','lon'])
nuevo = nuevo.loc[(nuevo.lat > -35) & (nuevo.lat < -34) & (nuevo.lon > -58.7)] #preparo para calcular distancias
nuevo['lat-lon'] = [(x, y) for x,y in zip(nuevo['lat'], nuevo['lon'])] #preparo para calcular distancias
#gp.scatter(caba_para_mapa['lat'], caba_para_mapa['lon'],colors='blue') #reviso si filtre bien "puntos malos"

In [11]:
from haversine import haversine
def distancia_minima(partida, points):
    dist_min = 9999999
    distancias=[]
    for destino in points:
        dist= round(haversine(partida,destino),1)
        if (dist<dist_min):
            dist_min = dist
    return dist

In [15]:
subtes = pd.read_csv('datos/datasets/CABA/latlon/estaciones-de-subte.csv')
subtes['lat-lon'] = [(x, y) for x,y in zip(subtes['y'], subtes['x'])]
nuevo['distancia_subte'] = [distancia_minima(x, list(subtes['lat-lon'])) for x in nuevo['lat-lon']]

In [16]:
univ = pd.read_csv('datos/datasets/CABA/latlon/universidades.csv',sep=';',error_bad_lines=False)
univ['lat-lon']= [(x, y) for x,y in zip(univ['LAT'], univ['LNG'])]
nuevo['distancia_universidad'] = [distancia_minima(x, list(univ['lat-lon'])) for x in nuevo['lat-lon']]

In [17]:
nuevo['lon']=properati['lat-lon'].str.split(",").str[0]
nuevo['lat']=properati['lat-lon'].str.split(",").str[1]

In [18]:
nuevo.to_csv('datos/properati_2.csv',index=False)